In [1]:
from dateutil import parser
from dateutil import relativedelta

import pandas as pd

In [2]:
df = pd.read_csv("../data/cleaned_reviews.tsv", sep="\t")
df.head()

,rating,vote,verified,reviewTime,reviewerName,reviewText,summary,quaterDiff,voteSuccess
0,5,0,True,2014-07-14,Alvey,great read every issue,Five Stars,16,0.0
1,3,0,True,2014-06-24,Jeff A Milne,magazine great time technology magazine new st...,Great While it lasted,16,0.0
2,5,0,True,2014-06-01,P. R THOMPSON,ive read maximum pc mpc many year article tech...,For Both the Newbie and The Advanced PC User,17,0.0
3,5,0,True,2014-05-15,Earle Williams,order magazine grandson then 7 go 30 wasis dep...,Great Gift,17,0.0
4,4,0,True,2014-03-28,JJM,subscribed nook version year thoroughly enjoy ...,Decent magazine at a good price,17,0.0


In [3]:
df["reviewTime"] = df["reviewTime"].apply(parser.parse)

In [4]:
# Sorts accendingly
lastest_date = list(df["reviewTime"].sort_values())[-1]
lastest_date

Timestamp('2018-09-14 00:00:00')

In [5]:
def quater_diff(date) -> int:
    diff = relativedelta.relativedelta(lastest_date, date)
    quarters = diff.years * 4
    for month in range(1, diff.months + 1):
        if month % 3 == 0:
            quarters += 1

    return quarters


df["quaterDiff"] = df["reviewTime"].apply(quater_diff)
df["quaterDiff"].head()

0    16
1    16
2    17
3    17
4    17
Name: quaterDiff, dtype: int64

In [6]:
def calc_vote_success(month_diff: int, votes: int) -> float:
    soften_success = 2  # Helps soften the division jump
    return votes / (month_diff + soften_success)


df["voteSuccess"] = df[["quaterDiff", "vote"]].apply(
    lambda x: calc_vote_success(*x), axis=1
)

df.sort_values("voteSuccess").tail()

,rating,vote,verified,reviewTime,reviewerName,reviewText,summary,quaterDiff,voteSuccess
18802,5,802,True,2011-05-08,Stuart|Anne,elementary age kid begin understand large scop...,"Interesting, thoughtful articles for kids who ...",29,25.870968
1554,1,243,True,2017-01-28,FoothillsJim,phone 30 minute cancel subscription autorenewe...,"Impossible to cancel, Renews WITHOUT the PROMI...",6,30.375000
11332,2,280,True,2016-12-13,Richy Fukushima,feel slightly deceive magazine receive like sm...,misleading product,7,31.111111
1796,1,445,True,2015-08-24,AvidConsumer,amazon really need cycle high rat review every...,Ignore the glowing reviews from 10 years ago -...,12,31.785714
7105,5,560,True,2017-08-07,Sarah,love new yorker beware amazon lightning deal 5...,Beware Amazon's $5 for 12 issues 'deal',4,93.333333


In [7]:
df.to_csv("../data/cleaned_reviews.tsv", index=False, sep="\t")